In [2]:
from multiprocessing.pool import ThreadPool
import dask.dataframe as dd
import dask
dask.config.set(pool=ThreadPool(500))

In [2]:
df = dd.read_parquet('../data/merge/mp_current_2018.parquet')

In [3]:
# Normalizando para -180 a 180 longitude
normalize = lambda x: (x < -180) and (x + 360) or (x > 180) and x - 360 or x

df['lon'] = df['lon'].map(normalize)

In [4]:
#Criando Feat oceans
def get_ocean(row):
    if (-60 <= row['lat'] <= 69) and (-83 <= row['lon'] <= 20):
        return 'Atlantic'
    elif (-60 <= row['lat'] <= 58) and ((128 <= row['lon'] < 181) or (-180 <= row['lon'] <= -67)):
        return 'Pacific'
    elif (-60 <= row['lat'] <= 31) and (20 <= row['lon'] <= 146):
        return 'Indian'
    elif (51 <= row['lat'] <= 90) and (-180 <= row['lon'] <= 180):
        return 'Artic'
    elif (-85 <= row['lat'] <= -60) and (-180 <= row['lon'] <= 180):
        return 'Southern Ocean'
    elif (30 <= row['lat'] <= 45) and (6 <= row['lon'] <= 36):
        return 'Mediterranean'


df['ocean'] = df.apply(lambda row: get_ocean(row), axis=1)

/Users/vitorhugo/Desktop/facu.nosync/PER_4/EAM/TPs/main/venv/lib/python3.8/site-packages/dask/dataframe/core.py:5314: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'object'))

  warnings.warn(meta_warning(meta))


In [5]:
df.to_parquet('../data/merge/mp_current_new_feat.parquet')

(None,)

In [3]:
df_oceans = dd.read_parquet('../data/merge/mp_current_2018_add_oceans.parquet')
df_oceans = df_oceans[df_oceans['ocean'].isnull() != True]
df_oceans = df_oceans.drop(['latitude', 'longitude'], axis=1)

In [5]:
# df_oceans.to_parquet('../data/merge/mp_current_2018_add_oceans_no_null.parquet',compute=True)
df_oceans.columns

Index(['time', 'lat', 'lon', 'MP_concentration', 'stdev_MP_samples',
       'num_MP_samples', 'u', 'v', 'ug', 'vg', 'ocean'],
      dtype='object')